## Mini recordatorio: el Índice en pandas (2 min)

- El índice identifica filas y se usa para alinear datos. Puede ser numérico, de texto, fechas o `MultiIndex`.
- No tiene que ser único, pero si lo usas como clave de join, cuida duplicados para evitar multiplicación de filas.
- Muchas operaciones alinean por índice (p. ej., `concat(axis=1)` y `join`).
- Herramientas clave: `set_index(col)`, `reset_index()`, `rename_axis`, `sort_index()`.
- Regla práctica: define explícitamente tus llaves (columnas) y usa índice solo cuando tenga sentido semántico (p. ej., tiempo).


In [ ]:
import pandas as pd

print("Índice por defecto (0..n-1) y 'set_index' / 'reset_index':")
base = pd.DataFrame({
    "id": [101, 102, 103],
    "nombre": ["Ana", "Luis", "Mara"],
    "monto": [10, 20, 30],
})
print("DataFrame base:")
print(base)
print("Índice actual:", base.index)

print("\nEstablecemos 'id' como índice (set_index):")
by_id = base.set_index("id")
print(by_id)
print("Índice ahora es 'id':", by_id.index)

print("\nreset_index(): 'id' vuelve a ser columna y el índice se reenumera:")
print(by_id.reset_index())

print("\nMultiIndex con ['id','nombre'] (2 niveles de identificación):")
multi = base.set_index(["id", "nombre"])  # no lo usamos después; es ilustrativo
print(multi)
print("Niveles del índice:", multi.index.names)



# 01 - Joins, Concat y el papel del Índice

Objetivos:
- Entender `merge` (inner, left, right, outer) y cómo afectan las filas.
- Diferenciar `merge`, `join` y `concat`.
- Ver cómo el índice influye en joins y concatenaciones.

Contexto del notebook:
- Trabajaremos con `clientes` y `ventas`. La clave de unión será `cliente_id` (con duplicados para observar sus efectos).
- Veremos cómo el índice NO es la clave aquí, y por qué a veces conviene `reset_index()` antes de unir.

Sigue las celdas: primero leemos, luego ejecutamos. Cada bloque de código imprime qué está pasando.


In [1]:
import pandas as pd
import numpy as np

print("Configuración básica lista. Versiones:")
print("pandas=", pd.__version__)

# Pequeños DataFrames de ejemplo
clientes = pd.DataFrame({
    "cliente_id": [1, 2, 2, 4],  # 2 está duplicado a propósito
    "nombre": ["Ana", "Luis", "Luis_dup", "Mara"],
}).set_index(pd.Index([10, 11, 12, 13], name="row_id"))

ventas = pd.DataFrame({
    "cliente_id": [2, 3, 4, 4],
    "monto": [100, 200, 150, 50],
})

print("\nclientes (índice no es la clave, 'cliente_id' sí):")
print(clientes)
print("\nventas:")
print(ventas)


Configuración básica lista. Versiones:
pandas= 2.3.3

clientes (índice no es la clave, 'cliente_id' sí):
        cliente_id    nombre
row_id                      
10               1       Ana
11               2      Luis
12               2  Luis_dup
13               4      Mara

ventas:
   cliente_id  monto
0           2    100
1           3    200
2           4    150
3           4     50


## `merge`: tipos de join (qué incluyen y por qué)

Setting:
- Tablas: `clientes` y `ventas`.
- Clave: `cliente_id`. Hay duplicados para ilustrar multiplicación de filas.

Tipos de join y su efecto:
- `inner`: devuelve solo claves que aparecen en ambas tablas. Útil para quedarse con intersección (p. ej., clientes que sí compraron).
- `left`: conserva todas las filas del lado izquierdo y trae datos del derecho cuando hay match; si no, `NaN`. Útil para enriquecer un universo base.
- `right`: simétrico a `left` pero tomando como base el derecho.
- `outer`: unión de claves de ambos lados; rellena faltantes con `NaN`. Útil para auditorías o detectar huecos.

Consideraciones importantes:
- Duplicados en la clave en cualquiera de los lados → filas se multiplican (producto cartesiano de las coincidencias).
- Claves faltantes (nulos) no hacen match.
- `indicator=True` añade `_merge` con `left_only/right_only/both` para auditar qué filas vienen de dónde.

Usaremos `on='cliente_id'` y `indicator=True` para observar el comportamiento en cada `how=`.


In [2]:
for how in ["inner", "left", "right", "outer"]:
    print(f"\n=== merge how='{how}' por 'cliente_id' ===")
    m = pd.merge(clientes.reset_index(), ventas, on="cliente_id", how=how, indicator=True)
    print(m)

print("\nNota: el cliente_id=3 solo existe en ventas; el 1 solo en clientes; duplicados de claves multiplican filas.")



=== merge how='inner' por 'cliente_id' ===
   row_id  cliente_id    nombre  monto _merge
0      11           2      Luis    100   both
1      12           2  Luis_dup    100   both
2      13           4      Mara    150   both
3      13           4      Mara     50   both

=== merge how='left' por 'cliente_id' ===
   row_id  cliente_id    nombre  monto     _merge
0      10           1       Ana    NaN  left_only
1      11           2      Luis  100.0       both
2      12           2  Luis_dup  100.0       both
3      13           4      Mara  150.0       both
4      13           4      Mara   50.0       both

=== merge how='right' por 'cliente_id' ===
   row_id  cliente_id    nombre  monto      _merge
0    11.0           2      Luis    100        both
1    12.0           2  Luis_dup    100        both
2     NaN           3       NaN    200  right_only
3    13.0           4      Mara    150        both
4    13.0           4      Mara     50        both

=== merge how='outer' por 'clien

## Índices, `left_index`/`right_index`, `join` y `concat` (alineación)

- `merge` por índice: usar `left_index=True` y/o `right_index=True` cuando la clave de unión está en el índice. Útil con índices temporales.
- `DataFrame.join`: atajo para unir por índice (y opcionalmente `on=` para columna del izquierdo). Ideal cuando ya tienes el índice “listo”.
- `concat`:
  - `axis=0` apila filas (une esquemas; puede crear `NaN` si no hay columnas comunes). Usa `ignore_index=True` para reenumerar.
  - `axis=1` pega columnas alineando por índice. Si los índices no coinciden, verás `NaN` en los huecos.
  - `keys=[...]` crea un `MultiIndex` que conserva el origen; muy útil para auditar o comparar dataframes.

Regla práctica:
- Si tu “llave” vive en columnas, prefiere `merge(on=...)`.
- Si tu “llave” está claramente en el índice (fechas/identificadores únicos), `join` o `merge(..._index=True)` pueden ser más naturales.


In [ ]:
print("Ejemplo 1) merge por ÍNDICE: usamos el índice (row_id) como llave, NO 'cliente_id'.")
print("- Útil solo si el índice es realmente tu clave semántica. Aquí es demostrativo.")
izq = clientes[["nombre"]]
der = clientes[["cliente_id"]]
print(pd.merge(izq, der, left_index=True, right_index=True, how="inner"))

print("\nEjemplo 2) join por índice (atajo de merge por índice):")
print("- Alinea por índice y conserva todas las filas de la izquierda (how='left').")
print(izq.join(der, how="left"))

print("\nEjemplo 3) concat(axis=0): apilar filas. Con ignore_index=True reenumeramos el índice.")
print(pd.concat([ventas.iloc[:2], ventas.iloc[2:]], axis=0, ignore_index=True))

print("\nEjemplo 4) concat(axis=1, keys=...): pegar columnas ALINEANDO por índice + etiquetar origen.")
print(pd.concat([izq, der], axis=1, keys=["nombres", "ids"]))


merge por índice (falso ejemplo: alineando índices actuales):
          nombre  cliente_id
row_id                      
10           Ana           1
11          Luis           2
12      Luis_dup           2
13          Mara           4

join por índice:
          nombre  cliente_id
row_id                      
10           Ana           1
11          Luis           2
12      Luis_dup           2
13          Mara           4

concat por filas (axis=0, ignore_index=True):
   cliente_id  monto
0           2    100
1           3    200
2           4    150
3           4     50

concat por columnas con keys (rastrea origen):
         nombres        ids
          nombre cliente_id
row_id                     
10           Ana          1
11          Luis          2
12      Luis_dup          2
13          Mara          4
